![header](logos/logo.png)

<center><h2> Ecopath 40 Years - Ostend, 7 June 2024 </h2></center>

<center><h4> Analyzing the Copernicus Marine Service products with Python </h4></center>

# 1. Introduction

In the previous tutorial, we downloaded two different datasets from two different products of the Copernicus Data Store.

In this tutorial, we will focus on how to open, analyze, and extract information from the data we have downloaded. We will use Python and XArray, a package for working with labeled multi-dimensional arrays. When we need to produce a visual representation of our data, we will use Matplotlib, which is probably the most common Python library for drawing plots.

First, we must specify where the files containing the data are stored.

In [ ]:
from pathlib import Path

TEMPERATURE_DATA = Path('PUT HERE THE PATH OF YOUR FILE')
CHLOROPHYLL_DATA = Path('PUT HERE THE PATH OF YOUR FILE')

# 2. Access to the data

In [ ]:
import xarray

In [ ]:
temperature_dataset = xarray.open_dataset(TEMPERATURE_DATA)
temperature_dataset

There is a lot of information inside our files! In addition to the variable with the temperature (**thetao**), there are also the coordinates of the grid. For example, we can explore the depth levels

In [ ]:
temperature_dataset.depth

We can see that there are 141 levels. The first one is about 1 meter deep, and the last one is 5754 meters below sea level. You can get the values of this array by using `.values` and specifying which element you want. For example, if you want the 10th element

In [ ]:
temperature_dataset.depth.values[10]

If you don't specify an index, the  `values` method returns an array with *all the elements* of the depth

In [ ]:
temperature_dataset.depth.values

You can also specify a range, for example from 2 to 5. In this case, you will get an array with all the elements in the range (from element number 2, starting from 0, to element number 5, excluded)

In [ ]:
temperature_dataset.depth.values[2:5]

If you do not specify the values of the range, you will get all the values (just like when we called `values` without using brackets). It may seem unnecessary, but we will see later how we can exploit this fact

In [ ]:
temperature_dataset.depth.values[:]

Levels are interesting, but we are far more interested in the values of our variable (the temperature). Everything we have said about the depth array still applies, but to identify a point on the grid, we need four coordinates: the time step, the depth, the latitude, and the longitude.

In [ ]:
temperature_dataset.thetao.values[7, 0, 15, 12]

In [ ]:
print(
    f'The temperature on the sea surface (depth index = 0) '
    f'at time step {temperature_dataset.time.values[7]} and at coordinates '
    f'({temperature_dataset.latitude.values[15]}, {temperature_dataset.longitude.values[12]}) '
    f'is {temperature_dataset.thetao.values[7, 0, 15, 12]}'
)

# 3. Plotting the data

It's time to visualize the data within our dataset. We will need the Matplotlib package to accomplish this task. In the next cell, we assign a "shortcut" to matplotlib. It will be invoked simply by writing `plt`

In [ ]:
import matplotlib.pyplot as plt

To draw a 1D plot, simply call the `plt.plot` method by passing two 1D arrays. A good example is plotting the values of the temperature at a specific time step and on a specific point against the depth.

We already know how to obtain an array with all the depths (temperature_dataset.depth.values). To obtain a 1D array of the temperature, we will use the trick of using an empty range `[:]` on the depth, while specifying all the other indices.

In [ ]:
depths_array = temperature_dataset.depth.values
temperature_column = temperature_dataset.thetao.values[7, :, 15, 12]

plt.xlabel('Depth [m]')
plt.ylabel('Potential temperature [-]')
plt.plot(depths_array, temperature_column)

But this is not the typical way to visualize the data of a water column. You can easily change this by adjusting the axes

In [ ]:
plt.ylabel('Depth [m]')
plt.xlabel('Potential temperature [-]')
plt.gca().invert_yaxis()
plt.plot(temperature_column, depths_array)

To visualize a 2D map, instead, we can use the `plt.imshow` method. For example, we can generate a map of the surface at a specific timestep. This requires building a 2D array: we use again the empty range (`:`) but for two indices: the latitude and the longitude 

In [ ]:
plt.imshow(temperature_dataset.thetao.values[2, 0, :, :], origin='lower')

Can you tell why there are some white areas on the map? What happens if we increase the depth?

In [ ]:
plt.imshow(temperature_dataset.thetao.values[2, 50, :, :], origin='lower')

How can we tell if a specific cell is inside the model or not? Here we propose a method that is quite safe even if it is a bit cumbersome. It relies on the `numpy` library, that contains many methods for manipulating arrays

In [ ]:
import numpy

land_cells = numpy.ma.getmaskarray(temperature_dataset.thetao.to_masked_array())
land_cells

`land_cells` is a 4-th dimensional array. On the other side, we don't need the first axis (the one related with the time) because the mask does not change during the timesteps. Therefore, we can remove the first axis and keep only the mask of the first time step. Here we use the attribute `.shape` to check the dimensions of the new array

In [ ]:
cells_on_land = land_cells[0, :, :, :]
cells_on_land.shape

# 4. Time averages

In the previous cells, we drew a map for a single time step. Now, we want to produce a map of the average surface temperature for each year in our domain.

We start by selecting the data, for example, for the year 2012. Since each year has 12 months and our dataset starts from 2010, the year 2012 will be represented by the slice 24:36. Here we get the slice

In [ ]:
temperature_dataset.thetao[24:36]

A much more convenient way to select the data we need is to use the method `loc`, that allows us to slice based on the values of the axis. Look how easily we can select the data related to the year 2012

In [ ]:
temperature_dataset.thetao.loc["2012-01-01":"2012-12-31", :, :, :]

You can also combine the `loc` method to get the data of a particular year within a specific range of longitude, such as between 12° and 14° degrees

In [ ]:
temperature_dataset.thetao.loc["2012-01-01":"2012-12-31", :, :, 12:14]

Let's go back to our average. Now we know how to extract the data on the surface! For example

In [ ]:
temperature_dataset.thetao.loc["2012-01-01":"2012-12-31"][:, 0, :, :]

To compute the average, we use `numpy` which contains a function perfect for this task. In the function we specify the argument `axis = 0`,which means "we want to compute the average with respect to the time". If we had written "axis = 2", it would computed the average along the longitude.

In [ ]:
average_ssd_2012 = numpy.average(temperature_dataset.thetao.loc["2012-01-01":"2012-12-31"][:, 0, :, :], axis=0)
plt.imshow(average_ssd_2012, origin="lower")

Let's have some fun! For example, we can produce maps showing the surface average temperature for each year of our model:

In [ ]:
for i in range(2010, 2022):
    data = numpy.average(temperature_dataset.thetao.loc[f"{i}-01-01":f"{i}-12-31"][:, 0, :, :], axis=0)
    plt.imshow(data, origin="lower")
    plt.title(f'Year {i}')
    plt.colorbar(label='Potential temperature [-]')
    plt.show()

Another example: can you guess what the purpouse of the following code is?

In [ ]:
for d in [1, 10, 50, 100, 500]:
    data = numpy.average(temperature_dataset.thetao.sel(depth=d, method='nearest'), axis=0)
    plt.imshow(data, origin="lower")
    plt.colorbar(label='Potential temperature [-]')
    plt.show()

# 5. Spatial averages

In this section, we focus on computing spatial averages. For example, we want to compute the average temperature for all the cells of the first time step of the model. If you have followed the previous parts of this tutorial, you might be thinking about doing something like this

In [ ]:
numpy.average(temperature_dataset.thetao.values[0, :, :, :])

We need to remove the cells for which we do not have data. For this task, we can use our vector `cells_on_land`. We can generate a vector that is `True` when a cell contains water (the opposite of `cells_on_land`) and then use it like a "mask" to retrieve only the values that we need.

In [ ]:
cells_on_water = numpy.logical_not(cells_on_land)
only_water_values = temperature_dataset.thetao.values[:, cells_on_water]
only_water_values.shape

It's worth noting that this new vector is a 2D array. The first index represents time, while the second represents the cell number. However, by doing this, we've lost information about the geometry, making it impossible to retrieve. Nevertheless, we can now compute the average of the values across the entire basin

In [ ]:
numpy.average(only_water_values, axis=1)

Unfortunately, the previous result is WRONG! It fails to consider the size of the cells. Although the model's grid is regular, meaning the coordinates of each cell's center are evenly spaced in terms of latitude and longitude, this does not ensure equal distances between them due to the Earth's shape. Consequently, the size of the cells varies, necessitating consideration when computing averages.

To address this, we'll approximate the area of a cell by multiplying the distance between its two sides, using the line passing through the center of the cell (refer to the following image, where each center is marked with a blue dot).

![header](images/grid.png)

Firstly, we need to compute the coordinates of the points that lie between the two centers, on the boundary of the cells. These points are the midpoints of the two centers.

In [ ]:
def midpoints(points):
    """
    Given a 1D array of points, compute the coordinates of
    the midpoint between each pair of adjacent points.
    In other words, `output[i]` represents the midpoint
    between `points[i]` and `points[i - 1]`.

    `output[0]` is chosen such that `points[0]` is the
    midpoint between `output[0]` and `output[1]`. The
    last midpoint is also computed following the same
    criteria.
    """
    n_points = points.shape[0]
    output = numpy.zeros(n_points + 1, dtype=points.dtype)
    for i in range(1, n_points):
        output[i] = (points[i] + points[i - 1]) / 2
    output[0] = 2 * points[0] - output[1]
    output[n_points] = 2 * points[n_points - 1] - output[n_points - 1]
    return output

In [ ]:
grid_latitude = midpoints(temperature_dataset.latitude)
grid_longitude = midpoints(temperature_dataset.longitude)

We now need to address the problem of computing the distance between two points described by their coordinates. While this problem can be solved manually, the result is often not accurate from a numerical standpoint. Instead, we will use a dedicated library (if you don't have it, you can install it with conda).

In [ ]:
from geopy.distance import geodesic

ostend = (51.225833, 2.919444)
brussels = (50.846667, 4.3525)

distance = geodesic(ostend, brussels)
print(f'The distance between Ostend and Brussels is {distance.km} km')

Now we have all the necessary components to compute the size of each cell. The result will be a 2D array containing the area in square kilometers. Please note that the algorithm is a bit complicated, so proceed with caution!

In [ ]:
def compute_cell_areas(center_latitude, center_longitude):
    grid_latitude = midpoints(center_latitude)
    grid_longitude = midpoints(center_longitude)

    cells_y = center_latitude.shape[0]
    cells_x = center_longitude.shape[0]

    cell_widths = numpy.zeros((cells_y, cells_x), dtype=numpy.float32)
    cell_heights = numpy.zeros((cells_y, cells_x), dtype=numpy.float32)

    # Now we compute the size of the horizontal lines that cross
    # every cell
    for x in range(cells_x):
        for y in range(cells_y):
            line_start = (center_latitude[y], grid_longitude[x])
            line_end = (center_latitude[y], grid_longitude[x + 1])
            line_length = geodesic(line_start, line_end).km
            cell_widths[y, x] = line_length

    # Now the same for the vertical ones
    for x in range(cells_x):
        for y in range(cells_y):
            line_start = (grid_latitude[y], center_longitude[x])
            line_end = (grid_latitude[y + 1], center_longitude[x])
            line_length = geodesic(line_start, line_end).km
            cell_heights[y, x] = line_length

    # Because cell_widths and cell_heights have the same shape, numpy
    # "broadcasts" the operation on each element of the two arrays
    cell_areas = cell_widths * cell_heights
    return cell_areas

In [ ]:
# Here we call the function to compute the cell_areas
cell_areas = compute_cell_areas(
    temperature_dataset.latitude,
    temperature_dataset.longitude
)

cell_areas

Now that we have the area of each cell, computing the volume only requires multiplying by the height of each level. Unfortunately, we do not have this information readily available. We need to compute it starting from the depth of the centers of the cells. Luckily, our `midpoints` function can provide us with the boundary of the levels, and then computing the height only requires subtracting the depth of two boundaries


In [ ]:
n_levels = temperature_dataset.depth.shape[0]
level_boundaries = middle_points(temperature_dataset.depth)

level_heights = numpy.zeros(n_levels, dtype=temperature_dataset.depth.dtype)

for i in range(n_levels):
    level_heights[i] = level_boundaries[i + 1] - level_boundaries[i]

level_heights

With `cell_areas` and `level_heights`, computing the volume of each cell is straightforward

In [ ]:
n_levels = temperature_dataset.depth.shape[0]
y_points = temperature_dataset.latitude.shape[0]
x_points = temperature_dataset.longitude.shape[0]

cell_volumes = numpy.zeros((n_levels, y_points, x_points), dtype=numpy.float32)
for i in range(n_levels):
    for j in range(y_points):
        for k in range(x_points):
            # we divide by 1000 so that the unit is km^3 (level_heights unit is meters)
            cell_volumes[i, j, k] = level_heights[i] / 1000 * cell_areas[j, k]

Finally, we can compute our average! We just need to remember to use our array `cells_on_water` to filter out the values outside the sea.

In [ ]:
# water_cell_volumes is a 1D array: we loose the information about the geometry
water_cell_volumes = cell_volumes[cells_on_water]

# This is the total volume of the basin (km^3)
total_volume = numpy.sum(water_cell_volumes)

# only_water_values is a 2D array: [time, cell_index]
only_water_values = temperature_dataset.thetao.values[:, cells_on_water]

average_values = numpy.zeros(temperature_dataset.time.shape[0])

for t in range(temperature_dataset.time.shape[0]):
    # The product inside the sum is a product of two 1D arrays (because we fix
    # the time index for only_water_values). numpy returns a 1D array whose
    # elements are the product of the elements of the input arrays
    average_values[t] = numpy.sum(only_water_values[t, :] * water_cell_volumes) / total_volume

plt.plot(temperature_dataset.time.values, average_values)

# 6. Spatial averages (exercise)

Now it's your turn! Utilize what you've learned to compute the average chlorophyll concentration ON THE SURFACE of the domain over time. Best of luck!

# 7. Extract values on the bottom layer (optional)

A common task you may be interested in is extracting the values at the bottom layer, or in other words, extracting the values that, when examining the data column by column, are in the last water cell before encountering a cell outside our domain.

A good approach to deal with this problem is to construct a 2D map that, for every column, gives us the index of the last water cell. We must decide what to do when we encounter a column that does not have any water cells. In this case, I propose saving "-1" (please remember that 0 would be the index of the first cell in Python).

The algorithm we use is as follows: for each column, we start by setting the index of the bottom cell to -1. As we scroll through the column, every time we encounter a water cell, we update the bottom cell index to the current cell's position. As soon as we find a cell outside the domain, we stop our search and set the bottom cell index for that column in our table.

In [ ]:
def build_bottom_map(cells_on_water):
    n_levels = cells_on_water.shape[0]
    y_points = cells_on_water.shape[1]
    x_points = cells_on_water.shape[2]

    bottom_map = numpy.zeros((y_points, x_points), dtype=numpy.int32)
    
    for i in range(y_points):
        for j in range(x_points):
            current_column = cells_on_water[:, i, j]
            bottom_map[i, j] = -1
            for l in range(n_levels):
                if current_column[l] == True:
                    bottom_map[i, j] = l
                else:
                    break
    return bottom_map

build_bottom_map(cells_on_water)

There is also a "trick" that can be used to produce our map of the bottom cells. We can simply "count" the number of water cells in each column. Internally, Python represents True as 1 and False as 0. Therefore, by summing along the depth axis, we get the number of water cells. The index of the last water cell is this number minus 1

In [ ]:
def build_bottom_map_tricky(cells_on_water):
    bottom_map = numpy.sum(cells_on_water, axis=0) - 1
    return bottom_map

We can verify that the two functions yield the same results by using NumPy to check if all the elements of their output arrays are equal

In [ ]:
numpy.all(build_bottom_map(cells_on_water) == build_bottom_map_tricky(cells_on_water))

We can use our map to extract the values of the temperature (for example) on the bottom layer. The idea is to prepare a 2D map and then to copy the values that we need with two nested for loops

In [ ]:
def extract_bottom(variable_data, bottom_map):
    n_levels = variable_data.shape[0]
    y_points = variable_data.shape[1]
    x_points = variable_data.shape[2]

    bottom_var = numpy.zeros((y_points, x_points), dtype=variable_data.dtype)
    
    for i in range(y_points):
        for j in range(x_points):
            if bottom_map[i, j] == -1:
                bottom_var[i, j] = numpy.nan
            else:
                bottom_layer_index = int(bottom_map[i, j])
                bottom_var[i, j] = variable_data[bottom_layer_index, i, j]

    return bottom_var

bottom_map = build_bottom_map_tricky(cells_on_water)
bottom_temp = extract_bottom(temperature_dataset.thetao[0, : ,: ,:], bottom_map)

bottom_temp

Here, we can visualize the map we have produced. However, it is challenging to have a good physical intuition of the values displayed because this map is heavily influenced by the bathymetry

In [ ]:
plt.imshow(bottom_temp, origin="lower")
plt.colorbar()

It would be interesting to compare the previous plot with the bathymetry of the domain.

As an exercise, draw the plot of the bathymetry using what you have learned in the previous examples. The bathymetry can be well approximated by the values of the depth axis, but you need to extract the values from the bottom layer. Good luck!